# Spread of an infectious disease
In our project, we are looking at the spread of a contagious disease in an enclosed space. In this room there are a predefined number of beds with patients in them. First, a patient is infected. The patient can infect the other patients on each side of his bed. Now it can be simulated over an arbitrary period of time how the disease spreads. It is possible to change the conditions of spreading with the help of different parameters. Among other things, the size of the room, the probability of infection or the bed of the first infected person can be changed. It is also possible to include other things in the simulation. For example, it is possible to vaccinate patients or move beds every day. In the following we will go into more detail about each part of the program and at the end we will show some examples of its use.
In the following, we will first explain the meaning of each variable. Then we will take a closer look at all the functions used to simulate the spreading. Next, we will explain how we have animated the simulation and finally we will show examples of how the simulation can be used properly.

Explanation of the variable names:

    c:              Choice of the user which simulations he wants
    n:              Number of rows in the dormitory
    m:              Number of columns in the dormitory
    initial_bed:    Row and column of the initial bed with the disease
    p:              Probability of an infection, between 0 and 1
    k:              Duration of the illness in days
    repetitions:    The number of days you want to simulate
    v:              Number of people vaccinated per day
    bs:             Number of beds swapped per day
    incu:           Duration of the incubation period in days 

## Cellular Automata
In this section we will take a closer look at the different functions we use to simulate the spreading. 
Each field in the grid can have a different value indicating the current status. The following statuses are used:

    0:          Healthy
    1:          Healed
    -1:         Vaccinated
    > 1:        Sick
    > 1 + k:    Sick but not infectious (Incubation period)


First we initialize the grid and the desired size and set all fields to 0. Then we set the desired initial bed to sick and return the new grid.

In [74]:
import numpy as np
import random

def initialize_grid(n, m, initial_bed, k, incu):
    grid = np.zeros((n, m))
    row, col = initial_bed
    grid[row, col] = 1+k+incu
    return grid

Next we have a function to find and return all neighbors of a bed. (top, bottom, left, right)

In [75]:
def get_neighbors(grid, row, col):
    n, m = grid.shape
    neighbors = []
    if row > 0:
        neighbors.append((row - 1, col))  # Top neighbor
    if row < n - 1:
        neighbors.append((row + 1, col))  # Bottom neighbor
    if col > 0:
        neighbors.append((row, col - 1))  # Left neighbor
    if col < m - 1:
        neighbors.append((row, col + 1))  # Right neighbor
    return neighbors

Now we start with the normal method to simulate the spread on one day. We show the evolution of this function and will add more features to the function with each step. As parameters we get the old grid, the probability of infection and the duration of the disease.
First we ask for the size of the grid and make a copy. Then we go through each field individually using two for loops. Then we check if the current field is sick (value greater than 1), if yes we subtract one sick day and search for all neighbors. For each neighbor it is checked if it is still healthy (value is 0) and then a random number is generated. If the neighbor is healthy and the generated number is greater than the value p, the neighbor is infected (value 1 + duration of the disease). Finally the new grid is returned.

In [76]:
def simulate_spread_basic(grid, p, k):
    n, m = grid.shape
    new_grid = np.copy(grid)
    for i in range(n):
        for j in range(m):
            if grid[i, j] > 1:  # Check if the bed is infected or already recovered
                new_grid[i,j] -= 1               
                for neighbor in get_neighbors(grid, i, j):
                    if grid[neighbor] == 0 and random.uniform(0, 1) < p:
                        new_grid[neighbor] = 1+k #set the neighbor ill                
    return new_grid

Now we will add a function that can vaccinate a certain number of healthy people every day. After a vaccination, the respective bed can no longer be infected. The function has two parameters, the number of people to vaccinate and the grid. 
A while loop is used to generate random numbers and try to vaccinate them. When enough vaccinations have been distributed, the new grid is returned.

In [77]:
def vaccinate(v, new_grid):
    n, m = new_grid.shape
    vaccinated = 0
    while(vaccinated < v):
        randi = random.randint(0,n-1)
        randj = random.randint(0, m-1)
        if new_grid[randi, randj] == 0:
            new_grid[randi, randj] = -1
            vaccinated += 1

    return new_grid

Now we will add a function that will swap a certain number of beds with each other every day. The function has two parameters, the number of beds to swap and the grid. 
A While loop is used to generate random numbers to select two beds. Then the values of the two beds are exchanged. When enough beds have been swapped, the new grid is returned.

In [78]:
def bed_swap(bs, new_grid):
    n, m = new_grid.shape
    bedsSwaped = 0
    while(bedsSwaped < bs):
        randi1 = random.randint(0,n-1)
        randj1 = random.randint(0, m-1)
        randi2 = random.randint(0,n-1)
        randj2 = random.randint(0, m-1)
        tmpVal = new_grid[randi1, randj1]
        new_grid[randi1, randj1] = new_grid[randi2, randj2]
        new_grid[randi2, randj2] = tmpVal
        bedsSwaped += 1

    return new_grid

### Final Function
In the final function a parameter is added (incu = The incubation period). The first part of the function remains almost unchanged, with the difference that now it is counted how many healthy beds are left. In addition, before the neighbors of a sick bed can be infected, it is checked if the incubation period is over and if a bed is infected, the incubation period is also added to the value.
Then a check is made to see if there are enough healthy beds left to vaccinate. If not, the value of the beds to be vaccinated is decreased.
Then, if necessary, the two functions to vaccinate and swap beds are called and finally the new grid is returned.

In [79]:
def simulate_spread(grid, p, k, v, bs, incu):
    n, m = grid.shape
    uninfectedBeds = 0
    new_grid = np.copy(grid)
    for i in range(n):
        for j in range(m):
            if grid[i, j] == 0:
                uninfectedBeds += 1
            if grid[i, j] > 1:  # Check if the bed is infected or already recovered
                new_grid[i,j] -= 1               
                for neighbor in get_neighbors(grid, i, j):
                    if new_grid[i,j] <= 1+k:        #check if incubation time is over
                        if grid[neighbor] == 0 and random.uniform(0, 1) < p:
                            new_grid[neighbor] = 1+k+incu #set the neighbor ill  
                            uninfectedBeds -= 1 
    
    if uninfectedBeds < v:
        v = uninfectedBeds

    if v > 0:
        new_grid = vaccinate(v, new_grid)

    if bs > 0:
        new_grid = bed_swap(bs, new_grid)
    
    return new_grid


## Animation


This section will present you how we manage to show, animate and save the results we calculated within three different methods. The first one is an animation graph, which is showing day by day the spread of the grid. The color representation will be explained below with the simulation. Then, we have the second animation which is a graph representing day by day the evolution of the disease, it allowes us to analyse better what we see and how the thing is doing. The last animation is not as useful as the first two, but we included it because it was interesting to try. It is a 3d representation of the first animation, the Y value is now the number of days that are left for the person to heal. 

Here is the first function that makes the 2d representation :

In [80]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt


def generate_animation(grid_list, save_path='animation.mp4'):
    # Check for NaN and infinity values in grid_list and replace them with 0
    grid_list = [np.where(np.isnan(grid) | np.isinf(grid), 0, grid) for grid in grid_list]

    # Define custom colormap
    cmap = colors.ListedColormap(['Yellow', 'Grey', 'Green', 'Red'])

    # Define the color boundaries
    bounds = [-1.5, -0.5, 0.5, 1.5, 1e10]  # Replace np.inf with a large number
    norm = colors.BoundaryNorm(bounds, cmap.N)

    # Create the figure and axis
    fig, ax = plt.subplots(figsize=(8, 6))

    # Initialize the image with the first grid_array
    img = ax.imshow(grid_list[0], cmap=cmap, norm=norm)
    plt.colorbar(img, ticks=[-1, 0, 1, 2], ax=ax, extend='both')  # Update the colorbar function
    plt.title("Grid Array Visualization")

    # Update function for the animation
    def update(frame):
        img.set_array(frame)
        return [img]

    # Create the animation
    anim = animation.FuncAnimation(fig, update, frames=grid_list, interval=500, blit=True)

    # Save the animation
    writer = animation.FFMpegWriter(fps=20, metadata=dict(artist='Me'), bitrate=1800)
    anim.save(save_path, writer=writer)


    # Show the animation
    plt.show()

Next, we have the line plot representation : 

In [81]:
def plot_recurrence(grid_list):
    # Count the recurrence of each number in each grid
    recurrence_counts = []
    for grid in grid_list:
        unique, counts = np.unique(grid, return_counts=True)
        recurrence_counts.append(dict(zip(unique, counts)))

    # Create a line chart
    fig, ax = plt.subplots(figsize=(8, 6))

    # Define the number-label mapping
    number_label_mapping = {-1: "Vaccinated", 0: "Not concerned", 1: "Healthy"}

    for number, label in number_label_mapping.items():
        count_list = [recurrence[number] if number in recurrence else 0 for recurrence in recurrence_counts]
        ax.plot(count_list, label=label)

    # Handle the case for numbers less than 1
    count_list = [sum(recurrence[number] for number in recurrence if number > 1) for recurrence in recurrence_counts]
    ax.plot(count_list, label="Ill")

    ax.set_xlabel('Days')
    ax.set_ylabel('Number of people')
    ax.set_title('Population State')
    ax.legend()

    plt.show()

And finally, the "useless" graph, which is the 3d representation of the first graph : 

In [82]:
def generate_3d_animation(grid_list):
    # Check for NaN and infinity values in grid_list and replace them with 0
    grid_list = [np.where(np.isnan(grid) | np.isinf(grid), 0, grid) for grid in grid_list]

    # Define custom colormap
    cmap = colors.ListedColormap(['Yellow', 'Grey', 'Green', 'Red'])

    # Define the color boundaries
    bounds = [-1.5, -0.5, 0.5, 1.5, 1e10]  # Replace np.inf with a large number
    norm = colors.BoundaryNorm(bounds, cmap.N)

    # Create the figure
    fig = plt.figure(figsize=(8, 6))

    # Create the 3D axis
    ax = fig.add_subplot(111, projection='3d')

    # Get the X, Y coordinates for the grid
    X, Y = np.meshgrid(np.arange(grid_list[0].shape[1]), np.arange(grid_list[0].shape[0]))

    # Initialize the surface plot with the first grid_array
    Z = grid_list[0]
    colors_array = cmap(norm(Z.flatten())).reshape(Z.shape + (4,))
    surf = ax.plot_surface(X, Y, Z, facecolors=colors_array, rstride=1, cstride=1)

    plt.title("Grid Array Visualization (3D)")

    # Update function for the animation
    def update(frame):
        ax.clear()
        Z = frame
        colors_array = cmap(norm(Z.flatten())).reshape(Z.shape + (4,))
        surf = ax.plot_surface(X, Y, Z, facecolors=colors_array, rstride=1, cstride=1)
        return [surf]

    # Create the animation
    anim = animation.FuncAnimation(fig, update, frames=grid_list, interval=500, blit=False)

    # Show the animation
    plt.show()

## User Input
In this section we show the user input and the main method. The following five functions are pure user input and therefore we will not explain them in detail. The first function lets the user decide whether to simulate the normal spreading or to use the additional options. The second one asks for all the parameters that are always needed and the last three are called if you want to set the other options as well. 

In [83]:
def get_user_choice():
    c = int(input("Enter the number of your choice:\n1: Normal Execute\n2: Add Vaccination\n3: Add Bed Swap\n4: Add Incubation Period "))
    return c

def get_user_input():
    n = int(input("Enter the number of rows in the dormitory: "))
    m = int(input("Enter the number of columns in the dormitory: "))
    initial_bed = input("Enter the initial infected bed position (in the format 'row,column'): ")
    initial_bed = tuple(map(int, initial_bed.split(',')))
    p = float(input("Enter the probability of infection (0 < p < 1): "))
    k = int(input("Enter the duration of the disease (in days): "))
    repetitions = int(input("Enter the number of days you want to simulate: "))
    return n, m, initial_bed, p, k, repetitions

def get_user_input_vaccines():
    v = int(input("Enter the number of vaccines per day: "))
    return v

def get_user_input_bedSwap():
    bs = int(input("Enter the number of Bed Swaps you want to perform per day: "))
    return bs

def get_user_input_incubation():
    incu = int(input("Enter the duration of the incubation period (in days): "))
    return incu

In the main method the user is first asked which options he wants to configure, then all required parameters are requested from the user. When all parameters are received, the first grid is created. Then a list of grids is created and the first one is copied into the list. Then the spreading is simulated for each day and the newly obtained grids are added to the list. Finally, the list with all grids is displayed using the animation functions.

In [84]:
def main():
    c = get_user_choice()
    n, m, initial_bed, p, k, repetitions = get_user_input()
    incu = 0
    v = 0
    bs = 0
    
    if c > 1:
        v = get_user_input_vaccines()
        if c > 2:
            bs = get_user_input_bedSwap()
            if c > 3:
                incu = get_user_input_incubation()
        

    grid = initialize_grid(n, m, initial_bed, k, incu)
    grid_list = [grid.copy()]
    
    for _ in range(repetitions):
        grid = simulate_spread(grid, p, k, v, bs, incu)
        grid_list.append(grid.copy())

    generate_animation(grid_list)
    plot_recurrence(grid_list)
    generate_3d_animation(grid_list)

## Simulations

This section will show what we can do with our application. In order to show what is possible, we will tweak and play with every parameters. with this, we will be able to see what is the impact of each parameter on the disease and its spread. 
<br>
<br>

Here is the color code that appear on every video : <br>
    - <text style="color:grey;"><strong>Grey</strong></text> = untouched population<br>
    - <text style="color:red;"><strong>Red</strong></text> = ill population, only them can spread the disease<br>
    - <text style="color:green;"><strong>Green</strong></text> = immune population, the people that had the disease and are now healthy<br>
    - <text style="color:yellow;"><strong>Yellow</strong></text> = vaccined part of the population. They are immune to the disease and therfore cannot spread<br>


## 1. Normal spreading


<h4><strong>Simulation with 20% chance of spreading</strong></h4>
<text><strong>100x100</strong> beds -- starting bed <strong>50,50</strong> -- <strong>10</strong> days disease duration -- <strong>200</strong> days simulation
<br>
<br>
<br>
<div style="display: flex; align-items: center;">
    <div style="flex: 1;">
        <video src="videos/1_low_spread.mp4" controls title="Title" style="width: 600px; height: 450px;"></video>
    </div>
    <div style="flex: 1;">
        <img src="images/1_low_spread.png" alt="Image" style="width: 600px; height: 450px;">
    </div>
</div>
<br>
<br>
<br>
<text>As we can see with this first simulation, the peak of the ill persons arrive after around 135 days. After 123 days, there are more people that are concerned about the illness than persons untouched.</text>
<br>
<br>
<text>The spreading factor will only accelerate the things. If we put the spreading percentage higher, there will be a bigger peak of ill persons. This peak will also arrive faster. The day that represent this : people not concerned < ill persons will also arrive sooner. </text>

<h4><strong>Simulation with 90% chance of spreading</strong></h4>
<text><strong>100x100</strong> beds -- starting bed <strong>50,50</strong> -- <strong>10</strong> days disease duration -- <strong>200</strong> days simulation
<br>
<br>
<br>
<div style="display: flex; align-items: center;">
    <div style="flex: 1;">
        <video src="videos/1_high_spread.mp4" controls title="Title" style="width: 600px; height: 450px;"></video>
    </div>
    <div style="flex: 1;">
        <img src="images/1_high_spread.png" alt="Image" style="width: 600px; height: 450px;">
    </div>
</div>
<br>
<br>
<br>
<text>As we can see with this second simulation, the peak of the ill persons arrive way sooner, around 50 days. And after the same time, there are more people that are concerned by this desease than persons that are not. </text>
<br>
<br>
<text>If the spreading factor is smaller than the recovery time, we can have a scenario where the people heal faster than the disease spread. We could even see that in this scenario, the disease don't survive and some people will never know about the disease. Here is an example :</text>

<h4><strong>Simulation where the spreading is unefficient enough to contamine everybody : 7% chance of spreading</strong></h4>
<text><strong>100x100</strong> beds -- starting bed <strong>50,50</strong> -- <strong>10</strong> days disease duration -- <strong>200</strong> days simulation
<br>
<br>
<br>
<div style="display: flex; align-items: center;">
    <div style="flex: 1;">
        <video src="videos/1_unefficient_spread.mp4" controls title="Title" style="width: 600px; height: 450px;"></video>
    </div>
    <div style="flex: 1;">
        <img src="images/1_unefficient_spread.png" alt="Image" style="width: 600px; height: 450px;">
    </div>
</div>


## 2. Vaccination

Now introducing the vaccination system. The vaccination is a parameter that we can change in order to vaccinate x NON ALREADY INFECTED persons per day. The vaccinated people will reduce the spread as they can't get infected. The people that are vaccinated will appear as yellow dots on the video. 

For this example, we will compare with the first video, with a spread of 20% chance.
In a grid of 100 by 100, there are 10'000 persons to vaccinate. So if we vaccinate 10 persons per day, at the end of the 200 days, 20% of the population will be vaccinated.


<h4><strong>Simulation with 30 vaccinated persons per day</strong></h4>
<text><strong>100x100</strong> beds -- starting bed <strong>50,50</strong> -- <strong>10</strong> days disease duration -- <strong>200</strong> days simulation -- <strong>90%</strong> chances of spreading</text>
<br>
<br>
<br>
<div style="display: flex; align-items: center;">
    <div style="flex: 1;">
        <video src="videos/2_vaccine_spread.mp4" controls title="Title" style="width: 600px; height: 450px;"></video>
    </div>
    <div style="flex: 1;">
        <img src="images/2_vaccine_spread.png" alt="Image" style="width: 600px; height: 450px;">
    </div>
</div>
<br>
<br>
<br>
<text>The first thing we can say about vaccination is that it is really efficient. On 10'000 people, we decided to vaccinate only 30 persons a day. I know it is easy maths once you see it, but after only 100 days, 30% of the population is vaccinated and therefore cannot spread the disease further.</text>
<br>
<br>
<text>The second thing is that in this scenario, it didn't appear to really "slow" the spread. But it makes people being healthy 30% faster. Also, we can see that the vaccine is more and more efficient day by day. In the first day, we vaccine 30ppl out of 10'000, which correspond of only 0.3% of the population. But in the final days, because the spreading space is a fixed rectangle, we vaccine 30 people out of 500, which correspond to 6%. And this goes higher and higher until that the number of vaccines per day > the number of new person ill. Which is the reason why we have only vaccinated people on the edge of the map.</text>

## 3. Swap beds

Now introducing the swap beds. Naturally, this parameter does affect the speed of the spread a lot as it makes the spread "teleport" in random locations of the map. In order to see how fast it is, we will compare it to the first simple spread of 20% chance, without any vaccination. Normally, the peak of ill people should be higher and the people concerned should be faster.


<h4><strong>Simulation with 3 swaps per day</strong></h4>
<text><strong>100x100</strong> beds -- starting bed <strong>50,50</strong> -- <strong>10</strong> days disease duration -- <strong>200</strong> days simulation -- <strong>20%</strong> chances of spreading</text>
<br>
<br>
<br>
<div style="display: flex; align-items: center;">
    <div style="flex: 1;">
        <video src="videos/3_swap_beds.mp4" controls title="Title" style="width: 600px; height: 450px;"></video>
    </div>
    <div style="flex: 1;">
        <img src="images/3_swap_beds.png" alt="Image" style="width: 600px; height: 450px;">
    </div>
</div>
<br>
<br>
<br>
<text>We can see that the spread does go really faster. The concerned people become higher than the not concerned people after only 85 days, compared to 120 days with the similar spread without bed swaps. Also, the peak of people being ill at the same time went from 1000 to almost 1800. This can explain why people from around the world, during covid times, were limiting and paying a lot of attention with travelling by plane or travelling from cities to other cities. In Switzerland, they ban the transfer from people houses to their secondary houses or chalet during weeks.</text>
<br>
<br>
<text>In fact, the spread goes so fast in so many random directions that we can see that, sometimes, some lucky people are surrounded by ill people becoming healthy with various days of healthiness, which create a "shield" around them and therefore causing randomly grey dots everywhere.</text>

## 4. Incubation period

The incubation period of a disease refers to the time between exposure to the pathogen and the onset of symptoms. It is during this period that the pathogen is replicating in the host's body. The duration of the incubation period can vary widely depending on the disease, ranging from a few days to several years.
<br>
<br>
Understanding the incubation period helps in identifying the time of infection and making decisions regarding disease control. For example, during the COVID-19 pandemic, knowing that the incubation period is around 2 to 14 days has led to recommendations for self-quarantine for at least 14 days for individuals who may have been exposed to the virus. This helps prevent further transmission of the disease.

<h4><strong>Simulation with an incubation period of 5 days</strong></h4>
<text><strong>100x100</strong> beds -- starting bed <strong>50,50</strong> -- <strong>10</strong> days disease duration -- <strong>300</strong> days simulation -- <strong>20%</strong> chances of spreading -- <strong>3</strong> swap bed involved per day -- <strong>NO</strong> vaccination involved</text>
<br>
<br>
<br>
<div style="display: flex; align-items: center;">
    <div style="flex: 1;">
        <video src="videos/4_incubation_period.mp4" controls title="Title" style="width: 600px; height: 450px;"></video>
    </div>
    <div style="flex: 1;">
        <img src="images/4_incubation_period.png" alt="Image" style="width: 600px; height: 450px;">
    </div>
</div>
<br>
<br>
<br>
<text>We knew that the incubation would slow the process, but we never thought that it would slow it this much. In this simulation, we took exactly the same parameters that the swap beds one and added only 5 incubation days. And the results are really changing. The % of concerned people is more than 50% after 225 days compared to 85 without the incubation. That is a +265% ratio.</text>
<br>
<br>
<text>Regarding the peak of people ill at the same time, it went from 1800 to 1000, which is kind of like the first simulations without bed swap.</text>

## Conclusion
In this project, we learned a lot about the spread of disease. At the beginning, we thought that it was very easy to predict the spread. But as we wrote the first function with the probability of infection and room size, we quickly realized that things that didn't matter in our minds at first can make a huge difference. As we refined the function and added the ability to vaccinate people and swap beds, the simulation became more and more realistic. Adding the incubation period made a much bigger difference than we thought at the beginning.

Then, after we finished the simulation function and we could see the different spreads with the help of the animations, we started to play with different values. Now, for example, in the corona pandemic, we understand the role of lockdown much more clearly, because if we don't exchange beds, the disease spreads much less quickly and is easier to contain. The importance of vaccination has also been demonstrated in our simulation. Because you manage to protect many people from infection and thus they can not infect other people.

Finally, we can say that we were very satisfied with our topic and really enjoyed the simulation. We have made many new exciting discoveries and have also learned a new programming language. The cooperation worked wonderfully and we are very satisfied with the result of our work.
